In [1]:
import re
import json
import base64
import random
import joblib
import numpy as np

from pathlib import Path
from collections import defaultdict

### Select motions (uncomment below to get keys.txt\)

In [ ]:
# data_root = Path('/Users/nikita/Projects/diploma/data/KIT/')
# npz_files = list(map(str, data_root.rglob('*.npz')))
# npz_files = list(map(lambda x: x.replace('/Users/nikita/Projects/diploma/data/KIT/', '').replace('_poses.npz', ''), npz_files))

In [ ]:
# support_prefixes = "955 952 950 965 987 1346 1347 948 917 883".split()

# no_support_prefixes = (
#     "969 1226 1229 1297 1487 1717 1721 1747 752 674 576 "
#     "575 572 551 441 63/ 12/ 11/ 10/ 9/ 8/ 7/ 6/ 4/"
# ).split()


# support_keywords = """seesaw
# slope
# step_stones
# downstair
# beam
# upstair
# HinUndSchuhbinden
# ErschreckenderHerrBischof
# RadSchlagen
# Handstand
# Kniebeuge
# Aufstehen
# support
# streching
# lean
# kneel
# downstar
# inspect_shoe_sole
# beam
# Handstand05
# experiment
# crawl
# balancing
# 912_3
# with_handrail
# with_table
# push""".split('\n')

# support_keywords = list(map(lambda x: x.strip().lower(), support_keywords))

# no_support_keywords = """turn
# kick
# evasion
# parkour
# Kopfschulterkniefuss
# HighFive
# Salutieren
# drinking
# synchron
# PizzaDelivery
# castBox
# Drehung
# Dreischritte
# Bouncen
# conversation
# Punch
# bend
# mixing
# walk
# step_over_gap
# run
# Preparing_the_dough
# wave
# tennis
# stomp
# wash
# squat
# throw
# kick
# jump
# golf
# drinking
# bow
# Transfuse
# trial
# guitar
# drum
# shake
# chicken
# point
# cal03
# dry""".split('\n')

# no_support_keywords = list(map(lambda x: x.strip().lower(), no_support_keywords))

In [ ]:
# selected_keys = set()
# unselected_keys = set()

# for key in npz_files:
#     support_prefix = sum(key.startswith(prefix) for prefix in support_prefixes) > 0
#     no_support_prefix = sum(key.startswith(prefix) for prefix in no_support_prefixes) > 0
#     support_keyword = any(keyword in key.lower() for keyword in support_keywords)
#     no_support_keyword = any(keyword in key.lower() for keyword in no_support_keywords)

#     if no_support_prefix and support_prefix:
#         print(key)
#     if no_support_prefix:
#         selected_keys.add(key)
    
#     elif not support_prefix and not no_support_prefix:
#         if support_keyword:
#             pass
#             unselected_keys.add(key)
#         elif no_support_keyword:
#             selected_keys.add(key)
#         else:
#             print(key)
#     else:
#         unselected_keys.add(key)


# selected_keys = list(map(lambda x: f'0-KIT_{x.split('/')[0]}_{x.split('/')[1]}_poses', selected_keys))
# random.shuffle(selected_keys)


# train_keys = selected_keys[: int(len(selected_keys)*0.8)]
# test_keys = selected_keys[int(len(selected_keys)*0.8):]

# Path('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_test_keys.txt').write_text('\n'.join(test_keys))
# Path('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_train_keys.txt').write_text('\n'.join(train_keys))

### Drop motions with bad initial poses

In [19]:
init_pose_train = joblib.load('/Users/nikita/Projects/diploma/fullbody/data/initial_pose/initial_pose_train.pkl')
init_pose_test = joblib.load('/Users/nikita/Projects/diploma/fullbody/data/initial_pose/initial_pose_test.pkl')
kit_train_motion_dict = joblib.load('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_train_motion_dict.pkl')
kit_test_motion_dict = joblib.load('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_test_motion_dict.pkl')

kit_train_keys = list(kit_train_motion_dict.keys())
kit_test_keys = list(kit_test_motion_dict.keys())

In [21]:
len(init_pose_train)

1964

In [15]:
selected_kit_train_keys = []
selected_kit_test_keys = []
selected_kit_train_motion_dict = {}
selected_kit_test_motion_dict = {}
selected_init_pose_train = {}
selected_init_pose_test = {}

new_id = 0
for old_id, motion in init_pose_train.items():
    if not any(item is None for item in motion.values()):
        selected_kit_train_keys.append(kit_train_keys[old_id])
        selected_kit_train_motion_dict[kit_train_keys[old_id]] = kit_train_motion_dict[kit_train_keys[old_id]]
        selected_init_pose_train[new_id] = motion
        new_id += 1


new_id = 0
for old_id, motion in init_pose_test.items():
    if not any(item is None for item in motion.values()):
        selected_kit_test_keys.append(kit_test_keys[old_id])
        selected_kit_test_motion_dict[kit_test_keys[old_id]] = kit_test_motion_dict[kit_test_keys[old_id]]
        selected_init_pose_test[new_id] = motion
        new_id += 1

In [16]:
Path('/Users/nikita/Projects/diploma/fullbody/data/tmp/kit_test_keys.txt').write_text('\n'.join(kit_test_keys))
Path('/Users/nikita/Projects/diploma/fullbody/data/tmp/kit_train_keys.txt').write_text('\n'.join(kit_train_keys))
joblib.dump(init_pose_train, '/Users/nikita/Projects/diploma/fullbody/data/tmp/initial_pose_train.pkl')
joblib.dump(init_pose_test, '/Users/nikita/Projects/diploma/fullbody/data/tmp/initial_pose_test.pkl')
joblib.dump(kit_train_motion_dict, '/Users/nikita/Projects/diploma/fullbody/data/tmp/kit_train_motion_dict.pkl')
joblib.dump(kit_test_motion_dict, '/Users/nikita/Projects/diploma/fullbody/data/tmp/kit_test_motion_dict.pkl')

['/Users/nikita/Projects/diploma/fullbody/data/tmp/kit_test_motion_dict.pkl']

In [17]:
Path('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_test_keys.txt').write_text('\n'.join(selected_kit_test_keys))
Path('/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_train_keys.txt').write_text('\n'.join(selected_kit_train_keys))
joblib.dump(selected_init_pose_train, '/Users/nikita/Projects/diploma/fullbody/data/initial_pose/initial_pose_train.pkl')
joblib.dump(selected_init_pose_test, '/Users/nikita/Projects/diploma/fullbody/data/initial_pose/initial_pose_test.pkl')
joblib.dump(selected_kit_train_motion_dict, '/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_train_motion_dict.pkl')
joblib.dump(selected_kit_test_motion_dict, '/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_test_motion_dict.pkl')

['/Users/nikita/Projects/diploma/fullbody/data/dataset/kit_test_motion_dict.pkl']

In [22]:
len(selected_init_pose_train), \
len(selected_init_pose_test)

(1964, 500)